In [1]:
import pandas as pd
## get df from hugging face, same as HumanEvalPlusOrig.jsonl
df = pd.read_parquet("hf://datasets/evalplus/humanevalplus/data/test-00000-of-00001-5973903632b82d40.parquet")

In [2]:
df

,task_id,prompt,canonical_solution,entry_point,test
0,HumanEval/0,from typing import List\n\n\ndef has_close_ele...,\n\n sorted_numbers = sorted(numbers)\n ...,has_close_elements,\n\nimport numpy as np\n\ndef is_floats(x) -> ...
1,HumanEval/1,from typing import List\n\n\ndef separate_pare...,"\n\n cnt, group, results = 0, """", []\n f...",separate_paren_groups,\n\nimport numpy as np\n\ndef is_floats(x) -> ...
2,HumanEval/2,\n\ndef truncate_number(number: float) -> floa...,\n\n return number - int(number)\n\n,truncate_number,\n\nimport numpy as np\n\ndef is_floats(x) -> ...
3,HumanEval/3,from typing import List\n\n\ndef below_zero(op...,\n account = 0\n for operation in operat...,below_zero,\n\nimport numpy as np\n\ndef is_floats(x) -> ...
4,HumanEval/4,from typing import List\n\n\ndef mean_absolute...,\n mean = sum(numbers) / len(numbers)\n ...,mean_absolute_deviation,\n\nimport numpy as np\n\ndef is_floats(x) -> ...
...,...,...,...,...,...
159,HumanEval/159,"\ndef eat(number, need, remaining):\n """"""\n...",\n\n if need <= remaining:\n return ...,eat,\n\nimport numpy as np\n\ndef is_floats(x) -> ...
160,HumanEval/160,"\ndef do_algebra(operator, operand):\n """"""\...","\n\n exp = """"\n for i in range(len(opera...",do_algebra,\n\nimport numpy as np\n\ndef is_floats(x) -> ...
161,HumanEval/161,"\ndef solve(s):\n """"""You are given a string...","\n ans, has_letter = """", False\n for ch ...",solve,\n\nimport numpy as np\n\ndef is_floats(x) -> ...
162,HumanEval/162,"\ndef string_to_md5(text):\n """"""\n Given...","\n if text == """": return None\n import h...",string_to_md5,\n\nimport numpy as np\n\ndef is_floats(x) -> ...


## Instructions

1. Run the notebook to save `HumanEvalPlusOrig` into executable test code.
2. Run the generate_report.py to generate report for cpu time and memory usage



In [3]:


def create_executable_script(row, output_filename):
    encodings = [
        'utf-8',
        'utf-8-sig',
        'gbk',
        'gb2312',
        'ascii'
    ]
    for encoding in encodings:
        try:
            with open(output_filename, 'w',encoding=encoding) as f:
                f.write('# -*- coding: utf-8 -*-'+'\n')
                f.write('''import psutil
import os
import time
from functools import wraps
def measure_performance(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        process = psutil.Process(os.getpid())

        start_time = time.perf_counter()
        start_mem = process.memory_info().rss / 1024  # KB

        result = func(*args, **kwargs)
        end_time = time.perf_counter()
        end_mem = process.memory_info().rss / 1024

        time_used = end_time - start_time
        mem_used = end_mem - start_mem

        return time_used*1_000_000,mem_used

    return wrapper'''+'\n')
                f.write(row['prompt'] + '\n')
                
                solution = row['canonical_solution'].strip().split('\n')
                flag = 0
                for line in solution:
                    if flag ==0:
                        f.write('    ' + line + '\n')
                        flag=1
                    else:
                        f.write(line + '\n')
                
                f.write('\n')  
                
                f.write(row['test'] + '\n')
                f.write('@measure_performance\n')
                f.write('def run():\n')
                f.write(f'    check({row["entry_point"]})\n')
                
                f.write('if __name__ == "__main__":\n')
                f.write(f'    time_used,mem_used = run()\n')
                f.write(f'    print(time_used,mem_used)\n')
            return True
        except Exception as e:
            print(f"Error with {encoding} encoding: {str(e)}")
            continue

In [4]:

for i in range(len(df)):
    row = df.iloc[i]
    task_id = row['task_id']
    entry_point = row['entry_point']
    file_path = 'canonical_solution/'+entry_point+'.py'
    create_executable_script(row, file_path)
